In [ ]:
import pandas as pd
import json

In [ ]:
# JSON-Datei laden
with open('final_single_question_data.json', 'r') as f:
    data = json.load(f)

# Daten in ein DataFrame umwandeln
rows = []
for entry in data:
    #question_id = entry["question_id"]
    question = entry["question"]
    question_type = entry["type"]
    for answer in entry["answers"]:
        rows.append({
            #"question_id": question_id,
            "question": question,
            "type": question_type,
            "answer_text": answer["answer_text"],
            "answer_label": answer["answer_label"],
            "timestamp": answer["timestamp"]
        })

# DataFrame erstellen
df_single_select = pd.DataFrame(rows)

# DataFrame anzeigen
#print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'final_single_question_data.json'

In [ ]:
df_single_select

,question,type,answer_text,answer_label,timestamp
0,What type of customer are you?,SINGLE_SELECT,I'm a first-time buyer exploring your offerings.,New customer,2025-01-24T00:52:21.492617
1,What type of customer are you?,SINGLE_SELECT,This is my initial purchase from your company.,New customer,2025-01-24T00:52:21.492675
2,What type of customer are you?,SINGLE_SELECT,I'm a new customer looking for information.,New customer,2025-01-24T00:52:21.492687
3,What type of customer are you?,SINGLE_SELECT,I've never used your services before.,New customer,2025-01-24T00:52:21.492696
4,What type of customer are you?,SINGLE_SELECT,"I'm a brand-new customer, excited to try your ...",New customer,2025-01-24T00:52:21.492704
...,...,...,...,...,...
6925,What is the specific customer group you're tar...,SINGLE_SELECT,Our specific customer base is R&D personnel in...,R&D,2025-01-24T01:06:02.563679
6926,What is the specific customer group you're tar...,SINGLE_SELECT,We're aiming to reach R&D professionals workin...,R&D,2025-01-24T01:06:02.563685
6927,What is the specific customer group you're tar...,SINGLE_SELECT,The specific customer group is R&D scientists ...,R&D,2025-01-24T01:06:02.563691
6928,What is the specific customer group you're tar...,SINGLE_SELECT,Our focus is on R&D teams working on national ...,R&D,2025-01-24T01:06:02.563697


In [ ]:
df_single_select = df_single_select[['question', 'answer_text', 'answer_label']]

In [ ]:
df_single_select

,question,answer_text,answer_label
0,What type of customer are you?,I'm a first-time buyer exploring your offerings.,New customer
1,What type of customer are you?,This is my initial purchase from your company.,New customer
2,What type of customer are you?,I'm a new customer looking for information.,New customer
3,What type of customer are you?,I've never used your services before.,New customer
4,What type of customer are you?,"I'm a brand-new customer, excited to try your ...",New customer
...,...,...,...
6925,What is the specific customer group you're tar...,Our specific customer base is R&D personnel in...,R&D
6926,What is the specific customer group you're tar...,We're aiming to reach R&D professionals workin...,R&D
6927,What is the specific customer group you're tar...,The specific customer group is R&D scientists ...,R&D
6928,What is the specific customer group you're tar...,Our focus is on R&D teams working on national ...,R&D


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
df_single_select["label"] = label_encoder.fit_transform(df_single_select["answer_label"])  # z. B. "Yes" → 0, "No" → 1, "No Preference" → 2

print(label_encoder.classes_)  # Zeigt, welche Zahl welchem Label entspricht

['1-10' '1-5' '11-15' '11-50' '16-20' '201-2000' '21-30' '31-40' '51-200'
 '6-10' 'Adito' 'Aerospace' 'Applicant' 'Automotive' 'CAS' 'Call'
 'Close.io' 'Computers & Networks' 'Construction company'
 'Consultant, Planner, Architect' 'Craft enterprises' 'Defense'
 'Education sector' 'End User' 'English' 'Existing customer' 'German'
 'Government' 'HubSpot' 'Industrial' 'Italian' 'Japanese ' 'Medical'
 'Meeting' 'Microsoft Dynamics' 'Network Operators & Infrastructure'
 'New customer' 'No' 'Offer' 'Partner' 'Physical Security' 'Pipedrive'
 'Production company' 'Public Safety / Law Enforcement' 'R&D'
 'SAP Sales Cloud' 'Salesforce' 'Satisfied' 'Scaffolding company'
 'Spanish' 'Trading company' 'Unsatisfied' 'Very satisfied'
 'Very unsatisfied' 'Wholesaler, Distributor' 'Yes' 'larger than 2000'
 'more than 40']


<ipython-input-39-ddba756f3a25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_single_select["label"] = label_encoder.fit_transform(df_single_select["answer_label"])  # z. B. "Yes" → 0, "No" → 1, "No Preference" → 2


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df, test_df = train_test_split(df_single_select, test_size=0.2, random_state=42, stratify=df_single_select["label"])

In [ ]:
# Speichere die Datasets als JSON
train_df[["question", "answer_text", "label"]].to_json("train.json", orient="records", lines=True)
test_df[["question", "answer_text", "label"]].to_json("test.json", orient="records", lines=True)

In [ ]:
print("Train size:", len(train_df), "Test size:", len(test_df))

Train size: 5544 Test size: 1386


In [ ]:
# pip install datasets

In [9]:
from datasets import load_dataset

ModuleNotFoundError: No module named 'datasets'

In [ ]:
dataset = load_dataset("json", data_files={"train": "train.json", "test": "test.json"})
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer_text', 'label'],
        num_rows: 5544
    })
    test: Dataset({
        features: ['question', 'answer_text', 'label'],
        num_rows: 1386
    })
})


In [ ]:
from transformers import AutoTokenizer

In [ ]:
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    text_inputs = [q + " " + a for q, a in zip(examples["question"], examples["answer_text"])]
    return tokenizer(text_inputs, truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/5544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1386 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
num_labels = len(label_encoder.classes_)  # Anzahl der Klassen (3)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./roberta_classification",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Use the One Cycle Policy
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    lr_scheduler_type="linear",  # 'linear' or 'cosine'
    warmup_steps=500,  # Steps to gradually increase the learning rate
    max_grad_norm=1.0, # Gradient clipping
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Funktion zur Berechnung der Metriken
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)  # Nimmt die Klasse mit der höchsten Wahrscheinlichkeit
    acc = accuracy_score(labels, predictions)  # Accuracy
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")  # Andere Scores
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics  # Hier fügen wir die Metrik-Funktion hinzu!
)


trainer.train()


<ipython-input-56-fd3cbb955914>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.159200,0.146767,0.959596,0.951587,0.959596,0.953841
2,0.087800,0.115750,0.975469,0.976074,0.975469,0.975446
3,0.072800,0.107475,0.976912,0.978866,0.976912,0.976602


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=2079, training_loss=0.6596670270005547, metrics={'train_runtime': 495.507, 'train_samples_per_second': 33.566, 'train_steps_per_second': 4.196, 'total_flos': 1094565840850944.0, 'train_loss': 0.6596670270005547, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./roberta_finetuned")
tokenizer.save_pretrained("./roberta_finetuned")

('./roberta_finetuned/tokenizer_config.json',
 './roberta_finetuned/special_tokens_map.json',
 './roberta_finetuned/vocab.json',
 './roberta_finetuned/merges.txt',
 './roberta_finetuned/added_tokens.json',
 './roberta_finetuned/tokenizer.json')

In [ ]:
label_mapping = {f"LABEL_{i}": label for i, label in enumerate(label_encoder.classes_)}
print(label_mapping)

{'LABEL_0': '1-10', 'LABEL_1': '1-5', 'LABEL_2': '11-15', 'LABEL_3': '11-50', 'LABEL_4': '16-20', 'LABEL_5': '201-2000', 'LABEL_6': '21-30', 'LABEL_7': '31-40', 'LABEL_8': '51-200', 'LABEL_9': '6-10', 'LABEL_10': 'Adito', 'LABEL_11': 'Aerospace', 'LABEL_12': 'Applicant', 'LABEL_13': 'Automotive', 'LABEL_14': 'CAS', 'LABEL_15': 'Call', 'LABEL_16': 'Close.io', 'LABEL_17': 'Computers & Networks', 'LABEL_18': 'Construction company', 'LABEL_19': 'Consultant, Planner, Architect', 'LABEL_20': 'Craft enterprises', 'LABEL_21': 'Defense', 'LABEL_22': 'Education sector', 'LABEL_23': 'End User', 'LABEL_24': 'English', 'LABEL_25': 'Existing customer', 'LABEL_26': 'German', 'LABEL_27': 'Government', 'LABEL_28': 'HubSpot', 'LABEL_29': 'Industrial', 'LABEL_30': 'Italian', 'LABEL_31': 'Japanese ', 'LABEL_32': 'Medical', 'LABEL_33': 'Meeting', 'LABEL_34': 'Microsoft Dynamics', 'LABEL_35': 'Network Operators & Infrastructure', 'LABEL_36': 'New customer', 'LABEL_37': 'No', 'LABEL_38': 'Offer', 'LABEL_39':

In [ ]:
test_df

,question,answer_text,answer_label,label
2474,What CRM system are you currently using?,We've recently transitioned to Adito for our CRM.,Adito,10
3120,What language should we use to communicate?,Using Italian gives a sense of connection to I...,Italian,30
4402,How many people work at your company?,Our workforce currently numbers between 51 and...,51-200,8
1824,What CRM system are you currently using?,Our team utilizes Pipedrive for CRM functional...,Pipedrive,41
5234,What industry are you in?,My government work centers on immigration and ...,Government,27
...,...,...,...,...
4186,How many people work at your company?,We're a tight-knit group of one to ten employees.,1-10,0
111,What type of customer are you?,I'm a new customer who is delighted with the o...,New customer,36
3009,What language should we use to communicate?,Our team’s expertise in German makes it the mo...,German,26
6590,What is the specific customer group you're tar...,Investors investing in businesses.,End User,23


In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model="./roberta_finetuned", tokenizer="./roberta_finetuned")

test_input = {
    "question": "What type of customer are you?",
    "answer_text": "first time here"
}

# Kombinieren wie im Training (Frage + Antwort)
test_text = test_input["question"] + " " + test_input["answer_text"]
result = classifier(test_text)
#print(result)

# Die vorhergesagte Klasse zurück in ein Label umwandeln
predicted_label = label_mapping[result[0]["label"]]
accuracy = result[0]["score"]

print(f"Vorhergesagtes Label: {predicted_label} (Accuracy: {accuracy})")


Device set to use cuda:0


Vorhergesagtes Label: New customer (Accuracy: 0.9964930415153503)


In [ ]:
import random

In [ ]:
from transformers import pipeline

# Initialize the classifier with the fine-tuned RoBERTa model and tokenizer
classifier = pipeline("text-classification", model="./roberta_finetuned", tokenizer="./roberta_finetuned")

question_list = df_single_select["question"].tolist()

# Interactive loop
while True:
    # Get user input for question and answer
    random_index = random.randint(0, len(question_list) - 1)
    question = question_list[random_index]
    print(f"Question: {question}")
    answer_text = input("Enter the answer text: ")

    # Combine question and answer as done in training
    test_text = question + " " + answer_text

    # Run the classifier on the combined text
    result = classifier(test_text)

    # Output the result
    print(f"Model result: {result}")

    # Assuming the result contains a class label (e.g., 'LABEL_0', 'LABEL_1', etc.), map it back to a label
    predicted_label = label_mapping.get(result[0]["label"], "Unknown label")

    # Print the predicted label
    print("Predicted label:", predicted_label)

    # Ask if the user wants to continue
    continue_input = input("Do you wan#t to test another input? (y/n): ")
    if continue_input.lower() != 'y':
        print("Exiting the interactive loop.")
        break


Device set to use cuda:0


Question: What kind of company is this?
Enter the answer text: tech
Model result: [{'label': 'LABEL_18', 'score': 0.7822200059890747}]
Predicted label: Construction company
Do you wan#t to test another input? (y/n): n
Exiting the interactive loop.


In [ ]:
import shutil

# Create a zip file for easier download
shutil.make_archive('/content/roberta_finetuned_model', 'zip', '/content', 'roberta_finetuned')

# The zip file is now ready to be downloaded
from google.colab import files
files.download("roberta_finetuned_model.zip")

'/content/roberta_finetuned_model.zip'

In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define the folder where the model was saved (this is the directory containing your model and tokenizer)
folder_to_zip = '/content/roberta_finetuned'  # The folder containing the fine-tuned model

# Specify the folder in Google Drive to save the model
model_save_path = 'model_path/roberta_finetuned'



# Copy model folder directly to Google Drive
shutil.copytree(folder_to_zip, model_save_path)
print(f"Model saved to Google Drive at {model_save_path}")


Mounted at /content/drive
Model saved to Google Drive at /content/drive/MyDrive/CapStone_models/roberta_finetuned


# Multi-Select Questions

In [1]:
import json
import pandas as pd

In [87]:
# JSON-Datei laden
with open('final_multi_question_data.json', 'r') as f:
    data = json.load(f)

# Daten in ein DataFrame umwandeln
rows = []
for row in data:
    question = row["question"]
    for ans in row["answers"]:
        rows.append({
            "question": question,
            "text": ans["answer_text"],
            "labels": ans["answer_label"].split(", "),  # Convert to list
            #"timestamp": ans["timestamp"]
        })

# DataFrame erstellen
df_multi_select = pd.DataFrame(rows)

# DataFrame anzeigen
#print(df.head())

In [113]:
df_multi_select

,question,text,labels
0,What are your product interests?,I'm interested in improving data quality and g...,"[DataQuality, BusinessCards]"
1,What are your product interests?,My priorities are data enrichment and visit re...,"[DataEnrichment, VisitReport]"
2,What are your product interests?,I need data cleansing and improved data quality.,"[Data Cleansing, DataQuality]"
3,What are your product interests?,I'm focused on business cards and data cleansing.,"[BusinessCards, Data Cleansing]"
4,What are your product interests?,My interest lies in visit reports and data enr...,"[VisitReport, DataEnrichment]"
...,...,...,...
740,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Jens Roschma...","['Stephan Maier', 'Joachim Wagner', 'Jens Rosc..."
741,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Domiki Stein...","['Stephan Maier', 'Joachim Wagner', 'Domiki St..."
742,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Sean Kennin ...","['Stephan Maier', 'Joachim Wagner', 'Sean Kenn..."
743,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Tim Persson ...","['Stephan Maier', 'Joachim Wagner', 'Tim Perss..."


In [89]:
question_label_mapping = {}

for _, row in df_multi_select.iterrows():
    question = row["question"]
    labels = row["labels"]

    # Ensure it's a list and update the mapping
    if question in question_label_mapping:
        question_label_mapping[question].update(labels)
    else:
        question_label_mapping[question] = set(labels)  # Use a set to avoid duplicates


In [90]:
question_label_mapping = {q: list(labels) for q, labels in question_label_mapping.items()}
question_label_mapping

{'What are your product interests?': ['DataEnrichment',
  'BusinessCards',
  'VisitReport',
  'Data Cleansing',
  'DataQuality'],
 'What problem are you trying to solve?': ['Capture trade fair contacts',
  'Improve CRM data quality',
  'Scan business cards',
  'Extract data from emails',
  'Clean up CRM'],
 'What type of contact is it?': ['Supplier',
  'New customer',
  'Competitor',
  'media',
  'Press',
  'Prospect',
  'Existing customer'],
 'What are you interested in?': ['300', '256', '234', '200', '100'],
 'When would you like a follow-up?': ['3 weeks', '1 week', '2 weeks'],
 'What products interest you?': ['High-speed interconnect testing',
  'Automotive radar target simulation',
  'Double-Pulse Testing',
  'Display port debugging and compliance',
  'Noise figure measurements'],
 'What products are you interested in?': ['JTS',
  'JS EcoLine',
  'MY-SYSTEM',
  'AX100',
  'Notion',
  'AKW100'],
 'What follow-up is planned?': ['No action',
  'Phone',
  'Email',
  'Schedule a Visit']

In [91]:
df_multi_select

,question,text,labels
0,What are your product interests?,I'm interested in improving data quality and g...,"[DataQuality, BusinessCards]"
1,What are your product interests?,My priorities are data enrichment and visit re...,"[DataEnrichment, VisitReport]"
2,What are your product interests?,I need data cleansing and improved data quality.,"[Data Cleansing, DataQuality]"
3,What are your product interests?,I'm focused on business cards and data cleansing.,"[BusinessCards, Data Cleansing]"
4,What are your product interests?,My interest lies in visit reports and data enr...,"[VisitReport, DataEnrichment]"
...,...,...,...
740,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Jens Roschma...","['Stephan Maier', 'Joachim Wagner', 'Jens Rosc..."
741,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Domiki Stein...","['Stephan Maier', 'Joachim Wagner', 'Domiki St..."
742,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Sean Kennin ...","['Stephan Maier', 'Joachim Wagner', 'Sean Kenn..."
743,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Tim Persson ...","['Stephan Maier', 'Joachim Wagner', 'Tim Perss..."


In [75]:
# Strip extra spaces and quotation marks from labels
df_multi_select['labels'] = df_multi_select['labels'].apply(lambda x: [label.strip().strip('\'') for label in x])

# List of all unique labels
all_labels = set(label for sublist in df_multi_select['labels'] for label in sublist)

# Initialize new columns for each label
for label in all_labels:
    df_multi_select[label] = df_multi_select['labels'].apply(lambda x: 1 if label in x else 0)

# Drop the 'labels' column as it is no longer needed
df_multi_select = df_multi_select.drop(columns=['labels'])


# Create a dictionary that maps labels to integers
label_to_int = {label: idx for idx, label in enumerate(all_labels)}

# Create a dictionary that maps integers back to labels
int_to_label = {idx: label for idx, label in enumerate(all_labels)}

In [92]:
df_multi_select

,question,text,labels
0,What are your product interests?,I'm interested in improving data quality and g...,"[DataQuality, BusinessCards]"
1,What are your product interests?,My priorities are data enrichment and visit re...,"[DataEnrichment, VisitReport]"
2,What are your product interests?,I need data cleansing and improved data quality.,"[Data Cleansing, DataQuality]"
3,What are your product interests?,I'm focused on business cards and data cleansing.,"[BusinessCards, Data Cleansing]"
4,What are your product interests?,My interest lies in visit reports and data enr...,"[VisitReport, DataEnrichment]"
...,...,...,...
740,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Jens Roschma...","['Stephan Maier', 'Joachim Wagner', 'Jens Rosc..."
741,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Domiki Stein...","['Stephan Maier', 'Joachim Wagner', 'Domiki St..."
742,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Sean Kennin ...","['Stephan Maier', 'Joachim Wagner', 'Sean Kenn..."
743,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Tim Persson ...","['Stephan Maier', 'Joachim Wagner', 'Tim Perss..."


In [78]:
#int_to_label

In [93]:
# !pip install datasets
from datasets import Dataset

In [94]:
dataset_df_multi_select = Dataset.from_pandas(df_multi_select)
dataset_df_multi_select

Dataset({
    features: ['question', 'text', 'labels'],
    num_rows: 745
})

In [96]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

In [97]:
question_label_mapping.get("What are your product interests?", [])

['DataEnrichment',
 'BusinessCards',
 'VisitReport',
 'Data Cleansing',
 'DataQuality']

In [84]:
#question_label_mapping

In [120]:
# Load tokenizer
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# neu mit Berücksichtigung von validen labels

def tokenize_function_multi(examples):
    # Take a batch of texts and concatenate question and answer
    text = [q + " [SEP] " + a for q, a in zip(examples["question"], examples["text"])]

    # Encode them with padding and truncation
    encoding = tokenizer(text, padding=True, truncation=True, max_length=384)

    # Initialize a labels matrix with zeros (shape: batch_size x num_labels)
    labels_matrix = np.zeros((len(text), len(all_labels)))

    # Iterate over the batch of examples
    for idx, row in enumerate(examples['question']):
        # Get the valid labels for the current question
        valid_labels = question_label_mapping.get(row, [])

        # Get the labels that are associated with the current example
        labels_batch = examples["labels"]#[idx]
        #print(labels_batch)

        # For each valid label, check if it is in the labels of the current example
        for label in labels_batch:
            if label in valid_labels:
                label_id = all_labels.index(label)  # Find the index of the valid label
                labels_matrix[idx, label_id] = 1  # Set to 1 if the label is valid and present

    # Add the labels matrix to the encoding
    encoding["labels"] = labels_matrix.tolist()

    return encoding


In [26]:
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function_multi(examples):
  # take a batch of texts
  text = [q + " [SEP] " + a for q, a in zip(examples["question"], examples["text"])]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)

  # Erstelle eine One-Hot-Encoding-Matrix mit der gleichen Größe wie die Anzahl der Labels
  labels_matrix = np.zeros((len(text), len(all_labels)))  # Hier `all_labels` ist die gesamte Liste aller möglichen Labels

    # Filtere die Labels basierend auf der Frage
  for idx, row in examples.iterrows():
    question = row["question"]
    valid_labels = question_label_mapping.get(question, [])
    for label in row["labels"]:
      if label in valid_labels:
          label_id = label2id.get(label)
          if label_id is not None:
              labels_matrix[idx, label_id] = 1  # Setze 1 für das gültige Label

  # Füge das One-Hot-Label-Matrix-Array zur Tokenizer-Ausgabe hinzu
  encoding["labels"] = labels_matrix.tolist()




  ## add labels
  #labels_batch = {k: examples[k] for k in examples.keys() if k in all_labels}
  ## create numpy array of shape (batch_size, num_labels)
  #labels_matrix = np.zeros((len(text), len(all_labels)))
  ## fill numpy array
  #for idx, label in enumerate(all_labels):
  #  labels_matrix[:, idx] = labels_batch[label]

  #encoding["labels"] = labels_matrix.tolist()

  return encoding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [121]:
tokenized_dataset_multi_select = dataset_df_multi_select.map(tokenize_function_multi, batched=True, remove_columns=dataset_df_multi_select.column_names)


Map:   0%|          | 0/745 [00:00<?, ? examples/s]

In [122]:
example = tokenized_dataset_multi_select[50]
print(example.keys())

dict_keys(['labels', 'input_ids', 'attention_mask'])


In [102]:
#tokenizer.decode(example['labels'])

In [123]:
tokenized_dataset_multi_select = tokenized_dataset_multi_select.train_test_split(test_size=0.2)

In [124]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(all_labels), problem_type="multi_label_classification", id2label=int_to_label, label2id=label_to_int)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [125]:
from transformers import TrainingArguments, Trainer

In [126]:

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,  # Erhöhe die Epochenanzahl
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,  # Kleinere Batch Size für Evaluation
    warmup_steps=1000,  # Möglicherweise mehr Warmup Steps für bessere Stabilität
    weight_decay=0.001,  # Kleineren Wert für Weight Decay
    logging_dir='./logs',
    logging_steps=100,  # Häufigeres Logging
    evaluation_strategy='steps',  # Häufigere Evaluierung, z. B. nach jedem Schritt
    save_steps=500,  # Speichern nach jedem Schritt
    eval_steps=500,  # Evaluieren nach jedem Schritt
    learning_rate=5e-5,  # Lernrate anpassen
    lr_scheduler_type="cosine",  # Verwende den Cosine Scheduler
    report_to="none",  # Verhindere Berichterstattung an externe Dienste
    load_best_model_at_end=True,  # Lade das beste Modell am Ende
    metric_for_best_model="accuracy",  # Welche Metrik wird verwendet, um das beste Modell zu bestimmen
    greater_is_better=True  # Wenn die Metrik besser wird, speichern
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [127]:
#!pip install evaluate
import evaluate

In [128]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [129]:
# Step 5: Prepare Trainer with the dataset and training arguments
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_multi_select["train"],
    eval_dataset=tokenized_dataset_multi_select["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # If you want to compute metrics like accuracy, you can define a custom function here
)

<ipython-input-129-47a89c8ecaa2>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [130]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=190, training_loss=0.4799514569734272, metrics={'train_runtime': 1999.1165, 'train_samples_per_second': 1.491, 'train_steps_per_second': 0.095, 'total_flos': 67411951205280.0, 'train_loss': 0.4799514569734272, 'epoch': 5.0})

In [131]:
model.save_pretrained("./roberta_finetuned_multiselect")
tokenizer.save_pretrained("./roberta_finetuned_multiselect")

('./roberta_finetuned_multiselect/tokenizer_config.json',
 './roberta_finetuned_multiselect/special_tokens_map.json',
 './roberta_finetuned_multiselect/vocab.json',
 './roberta_finetuned_multiselect/merges.txt',
 './roberta_finetuned_multiselect/added_tokens.json',
 './roberta_finetuned_multiselect/tokenizer.json')

In [132]:
from transformers import pipeline

# Lade die Pipeline für Multi-Label-Klassifikation
classifier = pipeline("text-classification", model="./roberta_finetuned_multiselect", tokenizer="./roberta_finetuned_multiselect", framework="tf")

# Frage stellen
question = "What are your product interests?"

# Benutzerantwort eingeben
answer = input("Bitte gib deine Antwort auf die Frage ein: ")

# Kombiniere Frage und Antwort
input_text = question + " " + answer

# Vorhersage der Labels mit der Pipeline
predictions = classifier(input_text)

# Ausgabe der Vorhersage
print("Vorhersage der Labels:", predictions)


All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Device set to use 0


Bitte gib deine Antwort auf die Frage ein: data quality
Vorhersage der Labels: [{'label': 'Jessica Hanke', 'score': 0.273768812417984}]


AttributeError: 'list' object has no attribute 'logits'

Ab hier alt

In [ ]:
# 🔹 Step 3: Clean Labels
mlb = MultiLabelBinarizer()
df_multi_select['labels'] = mlb.fit_transform(df_multi_select['labels']).tolist()  # Convert to binary format

In [ ]:
df_multi_select

,question,text,labels
0,What are your product interests?,I'm interested in improving data quality and g...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,What are your product interests?,My priorities are data enrichment and visit re...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,What are your product interests?,I need data cleansing and improved data quality.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,What are your product interests?,I'm focused on business cards and data cleansing.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,What are your product interests?,My interest lies in visit reports and data enr...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
740,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Jens Roschma...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
741,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Domiki Stein...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
742,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Sean Kennin ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
743,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Tim Persson ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, ..."


In [ ]:
import numpy as np

In [ ]:
for index, row in df_multi_select.iterrows():
    labels = row["labels"]
    labels = np.array(labels).astype(np.float32).tolist()
    df_multi_select.at[index, "labels"] = labels

df_multi_select

,question,text,labels
0,What are your product interests?,I'm interested in improving data quality and g...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,What are your product interests?,My priorities are data enrichment and visit re...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,What are your product interests?,I need data cleansing and improved data quality.,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,What are your product interests?,I'm focused on business cards and data cleansing.,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,What are your product interests?,My interest lies in visit reports and data enr...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
740,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Jens Roschma...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
741,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Domiki Stein...","[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
742,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Sean Kennin ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
743,Who should I CC on the follow-up?,"CC Stephan Maier, Joachim Wagner, Tim Persson ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."


In [ ]:
dataset_multiselect = Dataset.from_pandas(df_multi_select)

In [ ]:
dataset_multiselect

Dataset({
    features: ['question', 'text', 'labels'],
    num_rows: 745
})

In [ ]:
#!pip install -Uqq ipdb
import ipdb

In [ ]:
model_name = 'deepset/roberta-base-squad2'
tokenizer = RobertaTokenizer.from_pretrained(model_name)

# prompt: Step 4: Tokenization and other transformations roberta huggingface considering question and answer as input and label as output

def tokenize_function_multi(examples):
    # Use separator token to distinguish question & text
    text_inputs = [q + " [SEP] " + a for q, a in zip(examples["question"], examples["text"])]
    #ipdb.set_trace()
    tokenized_inputs = tokenizer(text_inputs, truncation=True, padding="max_length", max_length=128)
    tokenized_inputs["labels"] = [torch.tensor(label, dtype=torch.float) for label in examples["labels"]]


    return tokenized_inputs

In [ ]:
tokenized_dataset_multiselect = dataset_multiselect.map(tokenize_function_multi, batched=True)

Map:   0%|          | 0/745 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset_multiselect = tokenized_dataset_multiselect.train_test_split(test_size=0.2)

In [ ]:
num_labels = len(mlb.classes_)  # Number of unique labels
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, problem_type="multi_label_classification")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Step 4: Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',  # Evaluate at the end of each epoch
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
#!pip install evaluate

In [ ]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Stelle sicher, dass logits ein Tensor sind (falls sie als ndarray vorliegen)
    logits = torch.tensor(logits) if isinstance(logits, np.ndarray) else logits

    # Wende Sigmoid auf die Logits an, um Wahrscheinlichkeiten zu erhalten
    predictions = torch.sigmoid(logits).cpu().numpy()

    # Verwende einen Schwellenwert von 0.5, um Vorhersagen im multi-hot Format zu erhalten
    predictions = (predictions > 0.5).astype(int)

    # Konvertiere Labels zu NumPy-Array und stelle sicher, dass sie im richtigen Format sind
    labels = labels.cpu().numpy()

    # Berechne Accuracy und F1-Score
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')  # macro: F1 Score über alle Klassen hinweg

    return {'accuracy': accuracy, 'f1': f1}



In [ ]:

# Step 5: Prepare Trainer with the dataset and training arguments
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_multiselect["train"],
    eval_dataset=tokenized_dataset_multiselect["test"],
    tokenizer=tokenizer,
    compute_metrics=None,  # If you want to compute metrics like accuracy, you can define a custom function here
)

<ipython-input-210-d2d263528f90>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.445400,0.413896
2,0.391500,0.355573
3,0.319500,0.290695


TrainOutput(global_step=114, training_loss=0.3958316309410229, metrics={'train_runtime': 51.7145, 'train_samples_per_second': 34.574, 'train_steps_per_second': 2.204, 'total_flos': 117664496649216.0, 'train_loss': 0.3958316309410229, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./roberta_finetuned_multiselect")
tokenizer.save_pretrained("./roberta_finetuned_multiselect")

('./roberta_finetuned_multiselect/tokenizer_config.json',
 './roberta_finetuned_multiselect/special_tokens_map.json',
 './roberta_finetuned_multiselect/vocab.json',
 './roberta_finetuned_multiselect/merges.txt',
 './roberta_finetuned_multiselect/added_tokens.json')

In [ ]:
from transformers import pipeline

# Lade die Pipeline für Multi-Label-Klassifikation
classifier = pipeline("text-classification", model="./roberta_finetuned_multiselect", tokenizer="./roberta_finetuned_multiselect", framework="tf")

# Frage stellen
question = "What are your product interests?"

# Benutzerantwort eingeben
answer = input("Bitte gib deine Antwort auf die Frage ein: ")

# Kombiniere Frage und Antwort
input_text = question + " " + answer

# Vorhersage der Labels mit der Pipeline
predictions = classifier(input_text)

# Ausgabe der Vorhersage
print("Vorhersage der Labels:", predictions)


All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Device set to use 0


Bitte gib deine Antwort auf die Frage ein: data quality
Vorhersage der Labels: [{'label': 'LABEL_40', 'score': 0.29404422640800476}]
